In [ ]:
# Imports
# %matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# Reflect Tables into SQLAlchemy ORM

In [ ]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model & reflect the tables
base = automap_base()
base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
base.classes.keys()

In [ ]:
# Save references to each table
Measurement = base.classes.measurement
Station = base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

## Design a query to retrieve the last 12 months of precipitation data and plot the results


In [ ]:
# Calculate the date 1 year ago from the last data point in the database
last_date = session.query(Measurement.date).\
                order_by(Measurement.date.desc()).\
                first()
print(last_date)

In [ ]:
most_active = session.query(Measurement.station, func.count(Measurement.station)).\
        group_by(Measurement.station).\
        filter(func.strftime("%Y-%m-%d", Measurement.date)>= '2016-08-23').\
        order_by(func.count(Measurement.station).\
        desc())
station_most_active = [m for m in most_active[0]]
station_most_active[0]

In [ ]:
# Perform a query to retrieve the data and precipitation scores
prcp = session.query(Measurement.date, Measurement.prcp).\
                filter(func.strftime("%Y-%m-%d", Measurement.date)>= '2016-08-23').\
                all()

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
prcp = pd.DataFrame(prcp, columns=['Date','Precipitation'])
prcp = prcp.dropna(how='any')
prcp = prcp.reset_index(drop=True)
prcp = prcp.set_index('Date', inplace=False)

In [ ]:
# Sort the dataframe by date
prcp = prcp.sort_index()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
prcp.plot(rot=90)
plt.title('Precipitation of the last 12 months')
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
round(prcp.describe(),2)

In [ ]:
# Design a query to show how many stations are available in this dataset?
count_stations = session.query(Station.id).count()
count_stations

In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)? List in descending order.
active_stations = session.query(Measurement.station, func.count(Measurement.station)).\
                  group_by(Measurement.station).\
                  order_by(func.count(Measurement.station).desc())

for active in active_stations:
    print(active)

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, highest temperature recorded, and average temperature of the most active station?
most_active = session.query(func.min(Measurement.tobs),func.max(Measurement.tobs),func.avg(Measurement.tobs)).\
              filter(Measurement.station == 'USC00519281')

for active in most_active:
    print(f'Information on station USC00519281: \n - Lowest temperature: {active[0]} \n - Highest temperature: {active[1]} \n - Average temperature: {round(active[2],2)}')

In [ ]:
# Choose the station with the highest number of temperature observations.
for active in active_stations[:1]:
    most_station = active[0]
most_station

In [ ]:
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
toptemp_station = session.query(Measurement.date, Measurement.tobs).\
                filter(func.strftime("%Y-%m-%d", Measurement.date)>= '2016-08-23').\
                filter(Measurement.station == most_station).\
                all()
toptemp_station = pd.DataFrame(toptemp_station, columns=['Date','Temperature'])
toptemp_station = toptemp_station.dropna(how='any')
toptemp_station = toptemp_station.reset_index(drop=True)
toptemp_station = toptemp_station.set_index('Date', inplace=False)
toptemp_station = toptemp_station.sort_index()

In [ ]:
toptemp_station.plot(kind='hist')
plt.title('Temperatures of Station USC00519281 in the last 12 months')
plt.show()

## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax for your trip using the previous year's data for those same dates.
start_date = '2016-06-06'
end_date = '2016-06-19'
vac_data = calc_temps(start_date,end_date)
print(f'Information from {start_date} to {end_date}: \n - Lowest temperature: {vac_data[0][0]} \n - Highest temperature: {vac_data[0][1]} \n - Average temperature: {round(vac_data[0][2],2)}')

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
plt.figure(figsize=(1.5, 5), dpi=70)
plt.bar(0,vac_data[0][2],yerr=(vac_data[0][0]-vac_data[0][1]))
plt.title('Trip Avg Temp')
plt.ylabel('Temperature')
plt.tick_params(axis='x',which='both',bottom=False,top=False,labelbottom=False)
plt.show()

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
ly_start_date = '2015-06-06'
ly_end_date = '2015-06-19'

vac_rain = session.query(Station.station, Station.name, Station.latitude, Station.longitude, Station.elevation, Measurement.prcp).\
           filter(Measurement.date >= ly_start_date).\
           filter(Measurement.date <= ly_end_date).\
           filter(Measurement.station == Station.station).\
           group_by(Measurement.station).\
           order_by(Measurement.prcp)

rain = pd.DataFrame(vac_rain, columns=('Station','Station Name','Latitude','Longitude','Elevation','Precipitation'))
rain = rain.dropna(how='any')
rain = rain.reset_index(drop=True)
rain = rain.set_index('Station', inplace=False)

rain

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
